# Setup

In [1]:
from pathlib import Path
import numpy as np
import gensim

base_path = Path("go-full")

In [2]:
wv = gensim.models.KeyedVectors.load(str(base_path / "owl2vec" / "ontology.embeddings"), mmap="r")

In [2]:
import logging
from rich.logging import RichHandler

logging.basicConfig(level="INFO", format="%(message)s", handlers=[RichHandler()])

from pathlib import Path
import numpy as np
import gensim
from nltk import word_tokenize
import re

# Load the model
wv = gensim.models.KeyedVectors.load(str(base_path / "owl2vec" / "ontology.embeddings"), mmap="r")


# Helper functions from OWL2Vec_Plus.py
def pre_process_words(words):
    text = ' '.join([re.sub(r'https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE) for word in words])
    tokens = word_tokenize(text)
    processed_tokens = [token.lower() for token in tokens if token.isalpha()]
    return processed_tokens


# Load classes and annotations
classes = [line.strip() for line in open(base_path / "split" / "classes.txt").readlines()]
uri_label = {}

total_words = 0
for line in open(base_path / "cache" / "annotations.txt").readlines():
    tmp = line.strip().split()
    if tmp[1] == "preferred_label":
        uri_label[tmp[0]] = pre_process_words(tmp[2:])
        total_words += len(tmp) - 2

logging.info(f"Average number of words available per class: {total_words / len(classes)}")



n_s = []
lost_words = 0
def embed(model, instances, uri_label: dict):
    def word_embeding(inst):
        v = np.zeros(model.vector_size)
        if inst in uri_label:
            words = uri_label.get(inst)
            n = 0
            for word in words:
                if word in model.wv.index_to_key:
                    v += model.wv.get_vector(word)
                    n += 1
                else:
                    lost_words += 1
            return (v / n if n > 0 else v, n)
        else:
            return (v, 0)

    feature_vectors = []
    for instance in instances:
        # Get both URI and word embeddings
        # v_uri = model.get_vector(instance) if instance in model.index_to_key else np.zeros(model.vector_size)
        v_word, n = word_embeding(inst=instance)
        n_s.append(n)
        # Concatenate them
        feature_vectors.append(v_word)

    return feature_vectors


# Calculate embeddings
classes_e = embed(model=wv, instances=classes, uri_label=uri_label)
logging.info(f"Number of words lost: {lost_words}")
logging.info(f"Average number of words used per class: {np.mean(n_s)}")
# Extract GO IDs from class URIs and create dictionary
go_ids = [cls.split('/')[-1] for cls in classes]
embeddings_dict = {go_id: emb for go_id, emb in zip(go_ids, classes_e)}

# Save embeddings
np.save(str(base_path / "owl2vec" / "ontology.embeddings.npy"), embeddings_dict)

print(f"Saved embeddings for {len(embeddings_dict)} classes")
print(f"Each embedding has dimension: {len(next(iter(embeddings_dict.values())))} (word embeddings)")

[11/14/24 17:42:46] INFO     loading KeyedVectors object from go-full/owl2vec/ontology.embeddings      ]8;id=241450;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=33710;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#481\481]8;;\

[11/14/24 17:42:47] INFO     loading wv recursively from go-full/owl2vec/ontology.embeddings.wv.* with ]8;id=387479;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=600606;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#515\515]8;;\
                             mmap=r                                                                                

                    INFO     loading vectors from go-full/owl2vec/ontology.embeddings.wv.vectors.npy   ]8;id=631752;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=887812;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     loading syn1neg from go-full/owl2vec/ontology.embeddings.syn1neg.npy with ]8;id=74987;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=95633;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#520\520]8;;\
                             mmap=r                                                                                

                    INFO     setting ignored attribute cum_table to None                               ]8;id=588649;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=23276;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#553\553]8;;\

[11/14/24 17:42:48] INFO     Word2Vec lifecycle event {'fname': 'go-full/owl2vec/ontology.embeddings', ]8;id=552285;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=573313;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#447\447]8;;\
                             'datetime': '2024-11-14T17:42:48.669560', 'gensim': '4.3.3', 'python':                
                             '3.9.20 | packaged by conda-forge | (main, Sep 30 2024, 17:48:00)                     
                             \n[Clang 17.0.6 ]', 'platform': 'macOS-15.1-arm64-arm-64bit', 'event':                
                             'loaded'}                                                                             

[11/14/24 17:42:51] INFO     Average number of words available per class: 5.2709990300678955       ]8;id=669550;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_91454/1141742409.py\1141742409.py]8;;\:]8;id=55893;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_91454/1141742409.py#62\62]8;;\

[11/14/24 17:42:53] INFO     Number of words lost: 0                                               ]8;id=608648;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_91454/1141742409.py\1141742409.py]8;;\:]8;id=698384;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_91454/1141742409.py#66\66]8;;\

                    INFO     Average number of words used per class: 4.110203685741998             ]8;id=487594;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_91454/1141742409.py\1141742409.py]8;;\:]8;id=834103;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_91454/1141742409.py#67\67]8;;\

Saved embeddings for 51550 classes
Each embedding has dimension: 100 (word embeddings)
